<a href="https://colab.research.google.com/github/francesco-vaccari/ProjectDL/blob/fra/Baseline_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!gdown 1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq
!tar -xf /content/refcocog.tar.gz
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [ ]:
import clip
import torch
import numpy as np
from torch.utils.data import DataLoader
from RefcocogDataset import RefcocogDataset

# General Variables
REFCOCOG_PATH = "refcocog"
CLIP_VISUAL_MODEL = "ViT-B/32" # this should be ViT-B/16

device = "cuda" if torch.cuda.is_available() else "cpu"

## DATASET

In [ ]:
_, preprocess = clip.load(CLIP_VISUAL_MODEL)

train_dataset = RefcocogDataset(REFCOCOG_PATH, split="train", transform=preprocess, tokenization=clip.tokenize)
val_dataset = RefcocogDataset(REFCOCOG_PATH, split="val", transform=preprocess, tokenization=clip.tokenize)
test_dataset = RefcocogDataset(REFCOCOG_PATH, split="test", transform=preprocess, tokenization=clip.tokenize)

train_dataloader = DataLoader(train_dataset, shuffle=False)
val_dataloader = DataLoader(val_dataset, shuffle=False)
test_dataloader = DataLoader(test_dataset, shuffle=False)

train_features, train_bbox = next(iter(train_dataloader))
train_dataset.showImage(train_features, train_bbox)
# print(train_dataset.getSentences(train_features))
# print(len(train_dataset))
# print(len(val_dataset))
# print(len(test_dataset))

## BASELINE

In [ ]:
def YoloBBoxes(img, modelYOLO):
    result = modelYOLO(img)
    # result.show()
    bbox = result.pandas().xyxy[0]
    bbox = bbox.reset_index()
    bbox["tconfidence"] = np.nan
    bbox["crop"] = np.nan
    return bbox

def CropImage(image, boxs):
    crops = []
    for index, row in boxs.iterrows():
        box = (
            row['xmin'],
            row['ymin'],
            row['xmax'],
            row['ymax'],
        )
        crop = image.crop(box)
        crops.append(crop)
        boxs.at[index, 'crop'] = crop
    return crops

def computeSimilarity(image, sentences, modelCLIP, preprocessCLIP):
    similarities = []
    for sent in sentences:
        with torch.no_grad():
            image_features = modelCLIP.encode_image(torch.unsqueeze(preprocessCLIP(image).to(device), dim=0)).float().to(device)
            text_features = modelCLIP.encode_text(sent[0]).float().to(device)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarities.append(text_features.cpu().numpy() @ image_features.cpu().numpy().T)
    return sum(similarities)/len(similarities)

def computeIntersection(fx1, fy1, fx2, fy2, sx1, sy1, sx2, sy2):
    dx = min(fx2, sx2) - max(fx1, sx1)
    dy = min(fy2, sy2) - max(fy1, sy1)
    if (dx>=0) and (dy>=0):
        area = dx*dy
    else:
        area = 0
    return area

def computeAccuracy(bboxes, index, label):
    intersection = computeIntersection(bboxes['xmin'][index], bboxes['ymin'][index], bboxes['xmax'][index], bboxes['ymax'][index], 
                                       label[0].item(), label[1].item(), label[0].item()+label[2].item(), label[1].item()+label[3].item())
    area1 = (bboxes['xmax'][index]-bboxes['xmin'][index])*(bboxes['ymax'][index]-bboxes['ymin'][index])
    area2 = label[2].item()*label[3].item()
    union = area1 + area2 - intersection
    return intersection / union

def baseline(loader, dataset, modelYOLO, modelCLIP, preprocessCLIP):
    n_samples = 0
    tot_accuracy = 0
    for data_features, data_bbox in loader:

        bboxes = YoloBBoxes(dataset.getImage(data_features), modelYOLO)
        crops = CropImage(dataset.getImage(data_features), bboxes)

        if len(crops) > 0:
            highest_similarity = 0
            index_pred = 0
            for i, crop in enumerate(crops):
                similarity = computeSimilarity(crop, data_features['sentences'], modelCLIP, preprocessCLIP)
                if similarity > highest_similarity:
                    highest_similarity = similarity
                    index_pred = i
        
            accuracy = computeAccuracy(bboxes, index_pred, data_bbox)
        else:
            accuracy = 0
        
        tot_accuracy += accuracy
        n_samples += 1
        print(f'Image {n_samples:^8}/{len(dataset):^8}\t{accuracy}')

    return tot_accuracy/n_samples

In [ ]:
modelYOLO = torch.hub.load('ultralytics/yolov5', 'yolov5x')
modelCLIP, preprocessCLIP = clip.load(CLIP_VISUAL_MODEL, device=device)

baseline(test_dataloader, test_dataset, modelYOLO, modelCLIP, preprocessCLIP)